<center Plant Disease Detection Using CNN</center>



In [ ]:
! git clone https://github.com/spMohanty/PlantVillage-Dataset

The **PlantVillage-Dataset** from GitHub, This dataset contains images of various plant diseases. **54K+ images**

Set the path to the root directory where the raw color images are stored.

In [ ]:
DATASET_ROOT = "/content/PlantVillage-Dataset/raw/color"

List the contents of the dataset directory to verify the download.

`head` shows only the first few folders (the classes)

In [ ]:
!ls $DATASET_ROOT | head

Install the necessary Python libraries for deep learning and data handling.

This ensures all dependencies are met for the rest of the code

In [ ]:
# !pip install tensorflow tensorflow-datasets matplotlib seaborn scikit-learn

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
# Import specific modules from Keras for building the neural network.
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = (224, 224)
batch_size = 16

datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_gen = datagen.flow_from_directory(
    DATASET_ROOT,
    target_size=img_size,
    batch_size=batch_size,
    subset="training",
    class_mode="categorical"
)

In [ ]:
val_gen = datagen.flow_from_directory(
    DATASET_ROOT,
    target_size=img_size,
    batch_size=batch_size,
    subset="validation",
    class_mode="categorical"
)

In [ ]:
num_classes = len(train_gen.class_indices)
print("Classes:", train_gen.class_indices)

Build Model (Transfer Learning)

Load a pre-trained EfficientNetB0 model.

`include_top=False` means we are only using the feature-extracting base,not the final classification layers.

Freeze the base model's layers so their weights are not updated during training.

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze base initially


Add a GlobalAveragePooling2D layer on top of the base model's output.

This averages the feature maps, preparing the data for the final classification layers.

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

Add a BatchNormalization layer to stabilize and speed up training.

In [ ]:
# Batch Normalisation
x = tf.keras.layers.BatchNormalization()(x)

Add a Dropout layer to randomly ignore some neurons, which helps prevent overfitting.

In [ ]:
# Dropout for regularization
x = tf.keras.layers.Dropout(0.4)(x)

Add a fully connected (Dense) layer with 256 neurons and ReLU activation.

In [ ]:
x = tf.keras.layers.Dense(256, activation="relu")(x)

Add another BatchNormalization layer.

In [ ]:
# Batch Normalisation after dense layer
x = tf.keras.layers.BatchNormalization()(x)

In [ ]:
x = tf.keras.layers.Dropout(0.4)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

Create a new Sequential model, which stacks layers one after another, Add convolutional layers and pooling layers to extract features from images, more complex features.

Flatten the output of the convolutional layers into a 1D vector, Add a fully connected layer with 128 neurons


Add the final output layer with `num_classes` neurons for classification.

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

Compile Model

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Data Augmentation (Second, more detailed attempt)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
import os
import shutil
import random

# Create train/val directories
train_dir = "/content/PlantVillage-Dataset/train"
val_dir = "/content/PlantVillage-Dataset/val"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
# Loop over each class folder
dataset_dir = "/content/PlantVillage-Dataset/raw/color"
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        random.shuffle(images)
        split = int(len(images) * 0.8)  # 80% train, 20% val

        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        # Move images
        for img in images[:split]:
            shutil.copy(os.path.join(class_path, img), train_class_dir)
        for img in images[split:]:
            shutil.copy(os.path.join(class_path, img), val_class_dir)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/PlantVillage-Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
val_generator = test_datagen.flow_from_directory(
    '/content/PlantVillage-Dataset/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
 136/1357 ━━━━━━━━━━━━━━━━━━━━ 1:01:21 3s/step - accuracy: 0.0869 - loss: 5.0509

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Save the trained Keras model
model.save('/content/crop_disease_model.h5')
print("Model saved as crop_disease_model.h5")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create a folder inside Google Drive
save_path = "/content/drive/MyDrive/CropDiseaseModel"
import os
os.makedirs(save_path, exist_ok=True)

# Save Keras model (.h5)
model.save(f"{save_path}/crop_disease_model.h5")

# Save TFLite model (.tflite)
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open(f"{save_path}/crop_disease_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model saved to Google Drive at:", save_path)
